In [30]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [57]:
import pandas as pd
import calendar
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import FuncPuc1 as puc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pickle
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import xgboost as xgb
from sklearn.ensemble import ExtraTreesRegressor
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_excel('C:\\Users\\paulo.abreu\\Downloads\\archive\\Data_Train.xlsx')
len(df)

10683

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [4]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [5]:
puc.pre_p(df)
puc.feature_bin(df)

pre processamento concluído


C:\Users\paulo.abreu\Downloads\FuncPuc1.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dep_t_hour_bin'][i] = b
C:\Users\paulo.abreu\Downloads\FuncPuc1.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date_of_J_day_bin'][i] = f
C:\Users\paulo.abreu\Downloads\FuncPuc1.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lag_bin'][i] = h
C:\Users\paulo.abreu\Documents\Anaconda3\lib\site-packages\pandas\core\indexing.

Feature engineering concluído com sucesso


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Airline                10683 non-null  object        
 1   Date_of_Journey        10683 non-null  datetime64[ns]
 2   Source                 10683 non-null  object        
 3   Destination            10683 non-null  object        
 4   Route                  10682 non-null  object        
 5   Dep_Time               10683 non-null  object        
 6   Arrival_Time           10683 non-null  object        
 7   Duration               10683 non-null  object        
 8   Total_Stops            10682 non-null  object        
 9   Additional_Info        10683 non-null  object        
 10  Price                  10683 non-null  int64         
 11  Dep_time_hour          10683 non-null  float32       
 12  Date_of_Journey_day    10683 non-null  float32       
 13  D

In [7]:
df1 = df.copy()
df1 = df1.dropna()
df1 = df1.drop(['Date_of_Journey', 'Dep_Time' , 'Arrival_Time', 'Duration', 'Total_Stops'], axis =1)

#df1 = df1.drop(['Dep_time_slice','Arrival_Time_slice','Day_journey'], axis =1)


In [8]:
df1.head(2)

,Airline,Source,Destination,Route,Additional_Info,Price,Dep_time_hour,Date_of_Journey_day,Date_of_Journey_Month,today,lag,Dep_t_hour_bin,lag_bin,Date_of_J_day_bin
0,IndiGo,Banglore,New Delhi,BLR→DEL,No info,3897,22.0,24.0,March,2019-03-01,23,3.0,3.0,3
1,Air India,Kolkata,Banglore,CCU→IXR→BBI→BLR,No info,7662,5.0,1.0,May,2019-03-01,61,0.0,5.0,0


In [9]:
numeric_features = ['Dep_t_hour_bin', 'Date_of_J_day_bin', 'lag_bin'] #'Duration', 'Total_Stops'

categorical_features = ['Airline', 'Source' , 'Destination', 'Route', 'Additional_Info', 'Date_of_Journey_Month']


In [10]:
df_test = df1[0 : 5000]
df1 = df1[5000:]
df_test_y  = df_test['Price']
df_test_X = df_test.drop(['Price'], axis=1)


In [11]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [36]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regression', LinearRegression())])

In [38]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regression', ExtraTreesRegressor(random_state=42, n_jobs=-1))]) 

In [40]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regression', xgb.XGBRegressor(random_state=42, n_jobs=-1))])

In [55]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestRegressor(n_estimators=200, 
                                  random_state=10, 
                                    
                                  #class_weight= 'unbalanced', 
                                  max_features=0.2, 
                                  min_samples_leaf=1, 
                                  min_samples_split=2))])

In [59]:
y = df1['Price']
X = df1.drop(['Price'], axis =1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=0)

clf.fit(X_train, y_train)
print("Score do modelo (validação): %.3f" % clf.score(X_test, y_test))#X_test, y_test

print("Score do modelo (Teste): %.3f" % clf.score(df_test_X, df_test_y))#X_test, y_test

preds = clf.predict(df_test_X)

rmse = np.sqrt(mean_squared_error(df_test_y, preds))
print("RMSE: %f" % (rmse))

mae = np.sqrt(mean_absolute_error(df_test_y, preds))
print("MAE: %f" % (mae))
#clf.score(df_test_y, preds)

kfold = KFold(n_splits=10, random_state=7)

scores = cross_val_score(clf, df_test_X, df_test_y, cv=kfold)

print('Score Validação cruzada: %.3f (%.3f)' % (mean(scores), std(scores)))


Score do modelo (validação): 0.830
Score do modelo (Teste): 0.861
RMSE: 1561.348505
MAE: 27.648807
Score Validação cruzada: 0.867 (0.054)


In [ ]:
preds

In [ ]:
filename = 'modelo_final.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
modelo = pickle.load(open(filename, 'rb'))
result = modelo.score(df_test_X, df_test_y)
print(result)

In [ ]:
previsoes = modelo.predict(X)